## Predicting Coupon Redemption
XYZ Credit Card company regularly helps it’s merchants understand their data better and take key business decisions accurately by providing machine learning and analytics consulting. ABC is an established Brick & Mortar retailer that frequently conducts marketing campaigns for its diverse product range. As a merchant of XYZ, they have sought XYZ to assist them in their discount marketing process using the power of machine learning. Can you wear the AmExpert hat and help out ABC?

 
Discount marketing and coupon usage are very widely used promotional techniques to attract new customers and to retain & reinforce loyalty of existing customers. The measurement of a consumer’s propensity towards coupon usage and the prediction of the redemption behaviour are crucial parameters in assessing the effectiveness of a marketing campaign.

 
ABC’s promotions are shared across various channels including email, notifications, etc. A number of these campaigns include coupon discounts that are offered for a specific product/range of products. The retailer would like the ability to predict whether customers redeem the coupons received across channels, which will enable the retailer’s marketing team to accurately design coupon construct, and develop more precise and targeted marketing strategies.

 
The data available in this problem contains the following information, including the details of a sample of campaigns and coupons used in previous campaigns -

User Demographic Details
Campaign and coupon Details
Product details
Previous transactions
Based on previous transaction & performance data from the last 18 campaigns, predict the probability for the next 10 campaigns in the test set for each coupon and customer combination, whether the customer will redeem the coupon or not?

 



## Dataset Description
Here is the schema for the different data tables available. The detailed data dictionary is provided next.


![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/Screenshot-2019-09-28-at-8.58.32-PM.png)

## To summarise the entire process:

Customers receive coupons under various campaigns and may choose to redeem it.
They can redeem the given coupon for any valid product for that coupon as per coupon item mapping within the duration between campaign start date and end date
Next, the customer will redeem the coupon for an item at the retailer store and that will reflect in the transaction table in the column coupon_discount.
 

In [15]:
import pandas as pd

In [19]:
pd.read_clipboard()

,Variable,Definition
0,id,Unique id for coupon customer impression
1,campaign_id,Unique id for a discount campaign
2,coupon_id,Unique id for a discount coupon
3,customer_id,Unique id for a customer
4,redemption_status,"(target) (0 - Coupon not redeemed, 1 - Coupon ..."


In [18]:
pd.read_clipboard()

,Variable,Definition
0,campaign_id,Unique id for a discount campaign
1,campaign_type,Anonymised Campaign Type (X/Y)
2,start_date,Campaign Start Date
3,end_date,Campaign End Date


In [20]:
pd.read_clipboard()

,Variable,Definition
0,coupon_id,Unique id for a discount coupon (no order)
1,item_id,Unique id for items for which given coupon is ...


In [21]:
pd.read_clipboard()

,Variable,Definition
0,customer_id,Unique id for a customer
1,age_range,Age range of customer family in years
2,marital_status,Married/Single
3,rented,"0 - not rented accommodation, 1 - rented accom..."
4,family_size,Number of family members
5,no_of_children,Number of children in the family
6,income_bracket,Label Encoded Income Bracket (Higher income co...


In [22]:
pd.read_clipboard()

,Variable,Definition
0,date,Date of Transaction
1,customer_id,Unique id for a customer
2,item_id,Unique id for item
3,quantity,quantity of item bought
4,selling_price,Sales value of the transaction
5,other_discount,Discount from other sources such as manufactur...
6,coupon_discount,Discount availed from retailer coupon


In [23]:
pd.read_clipboard()

,Variable,Definition
0,item_id,Unique id for item
1,brand,Unique id for item brand
2,brand_type,Brand Type (local/Established)
3,category,Item Category


In [24]:
pd.read_clipboard()

,Variable,Definition
0,id,Unique id for coupon customer impression
1,campaign_id,Unique id for a discount campaign
2,coupon_id,Unique id for a discount coupon
3,customer_id,Unique id for a customer


In [1]:


import featuretools as ft
import pandas as pd
import numpy as np
from featuretools_tsfresh_primitives import AbsEnergy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
train=pd.read_csv("../data/train.csv")
compaign=pd.read_csv("../data/campaign_data.csv")
coupon_item_mapping=pd.read_csv("../data/coupon_item_mapping.csv")
customer_demographics=pd.read_csv("../data/customer_demographics.csv")
customer_transaction=pd.read_csv("../data/customer_transaction_data.csv")
item=pd.read_csv("../data/item_data.csv")
test=pd.read_csv("../data/test_QyjYwdj.csv")

In [3]:
train.head()
compaign.head()
coupon_item_mapping.head()
customer_demographics.head()
customer_transaction.head()
item.head()
test.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566
2,5,22,981,510
3,8,25,1069,361
4,10,17,498,811


In [4]:
#coupon_item_mapping=coupon_item_mapping.drop("coupon_index",axis=1)
#customer_transaction=customer_transaction.drop("transaction_index",axis=1)

In [5]:
es = ft.EntitySet(id = 'train')
es= es.entity_from_dataframe(entity_id = 'train', dataframe = train, 
                              index = 'id',variable_types={"campaign_id":ft.variable_types.Index})
es=es.entity_from_dataframe(entity_id="compaign",dataframe=compaign,index='campaign_id')
es=es.entity_from_dataframe(entity_id="coupon_item_mapping",dataframe=coupon_item_mapping,index="coupon_index",make_index=True)
es=es.entity_from_dataframe(entity_id="customer_demographics",dataframe=customer_demographics,index="customer_id")
es=es.entity_from_dataframe(entity_id="customer_transaction",dataframe=customer_transaction,index="transaction_index",make_index=True)
es=es.entity_from_dataframe(entity_id="item",dataframe=item,index="item_id")

es

Entityset: train
  Entities:
    train [Rows: 78369, Columns: 5]
    compaign [Rows: 28, Columns: 4]
    coupon_item_mapping [Rows: 92663, Columns: 3]
    customer_demographics [Rows: 760, Columns: 7]
    customer_transaction [Rows: 1324566, Columns: 8]
    item [Rows: 74066, Columns: 4]
  Relationships:
    No relationships

## Adding relationships between dataframes

In [6]:
train_compaign = ft.Relationship(es['compaign']['campaign_id'],es['train']['campaign_id'])
#es.add_relationship(train_compaign)
#train_coupon=ft.Relationship(es['train']['coupon_id'],es['coupon_item_mapping']["coupon_id"])
train_cust_demog=ft.Relationship(es['customer_demographics']['customer_id'],es['train']['customer_id'])

cust_demog_trans=ft.Relationship(es['customer_demographics']['customer_id'],es['customer_transaction']['customer_id'])
cust_trans_item=ft.Relationship(es['item']['item_id'],es['customer_transaction']['item_id'])
item_coupon=ft.Relationship(es['item']['item_id'],es['coupon_item_mapping']['item_id'])

In [7]:
es.add_relationships([train_compaign,train_cust_demog,cust_demog_trans,cust_trans_item,item_coupon])

Entityset: train
  Entities:
    train [Rows: 78369, Columns: 5]
    compaign [Rows: 28, Columns: 4]
    coupon_item_mapping [Rows: 92663, Columns: 3]
    customer_demographics [Rows: 760, Columns: 7]
    customer_transaction [Rows: 1324566, Columns: 8]
    item [Rows: 74066, Columns: 4]
  Relationships:
    train.campaign_id -> compaign.campaign_id
    train.customer_id -> customer_demographics.customer_id
    customer_transaction.customer_id -> customer_demographics.customer_id
    customer_transaction.item_id -> item.item_id
    coupon_item_mapping.item_id -> item.item_id

In [8]:
import matplotlib.pyplot as plt
#import os
#os.environ["PATH"]+='F:\anaconda\Library\bin\graphviz\dot.exe'
es.plot()

RuntimeError: To plot entity sets, a graphviz backend is required.
Install the backend using one of the following commands:
  Mac OS: brew install graphviz
  Linux (Ubuntu): sudo apt-get install graphviz
  Windows: conda install python-graphviz
  For more details visit: https://docs.featuretools.com/getting_started/install.html

In [13]:
es['train'].df.head()

,id,coupon_id,customer_id,redemption_status,campaign_id
1,1,27,1053,0,13
2,2,116,48,0,13
6,6,635,205,0,9
7,7,644,1050,0,13
9,9,1017,1489,0,8


In [15]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'train', 
max_depth = 2, 
verbose = 1)



Built 76 features
Elapsed: 00:04 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [17]:
feature_matrix.head()

,coupon_id,customer_id,redemption_status,campaign_id,compaign.campaign_type,customer_demographics.age_range,customer_demographics.marital_status,customer_demographics.rented,customer_demographics.family_size,customer_demographics.no_of_children,...,customer_demographics.MIN(customer_transaction.selling_price),customer_demographics.MIN(customer_transaction.other_discount),customer_demographics.MIN(customer_transaction.coupon_discount),customer_demographics.MEAN(customer_transaction.quantity),customer_demographics.MEAN(customer_transaction.selling_price),customer_demographics.MEAN(customer_transaction.other_discount),customer_demographics.MEAN(customer_transaction.coupon_discount),customer_demographics.COUNT(customer_transaction),customer_demographics.NUM_UNIQUE(customer_transaction.item_id),customer_demographics.MODE(customer_transaction.item_id)
id,,,,,,,,,,,,,,,,,,,,,
1,27,1053,0,13,X,46-55,NaN,0.0,1,NaN,...,17.45,-890.50,-89.05,340.487097,184.260484,-33.168935,-0.287258,310.0,208.0,14458.0
2,116,48,0,13,X,36-45,Married,0.0,2,NaN,...,7.12,-372.23,-213.72,31.540260,234.247013,-27.699169,-3.215039,385.0,244.0,23932.0
6,635,205,0,9,Y,46-55,Married,0.0,2,NaN,...,17.45,-468.05,-89.05,1.392784,121.094495,-17.795660,-2.212082,970.0,533.0,13294.0
7,644,1050,0,13,X,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1017,1489,0,8,X,46-55,Married,0.0,2,NaN,...,8.90,-320.22,-53.43,247.443060,120.636103,-20.524733,-0.471548,562.0,327.0,10673.0


In [18]:
feature_names

[<Feature: coupon_id>,
 <Feature: customer_id>,
 <Feature: redemption_status>,
 <Feature: campaign_id>,
 <Feature: compaign.campaign_type>,
 <Feature: customer_demographics.age_range>,
 <Feature: customer_demographics.marital_status>,
 <Feature: customer_demographics.rented>,
 <Feature: customer_demographics.family_size>,
 <Feature: customer_demographics.no_of_children>,
 <Feature: customer_demographics.income_bracket>,
 <Feature: compaign.SUM(train.coupon_id)>,
 <Feature: compaign.SUM(train.redemption_status)>,
 <Feature: compaign.STD(train.coupon_id)>,
 <Feature: compaign.STD(train.redemption_status)>,
 <Feature: compaign.MAX(train.coupon_id)>,
 <Feature: compaign.MAX(train.redemption_status)>,
 <Feature: compaign.SKEW(train.coupon_id)>,
 <Feature: compaign.SKEW(train.redemption_status)>,
 <Feature: compaign.MIN(train.coupon_id)>,
 <Feature: compaign.MIN(train.redemption_status)>,
 <Feature: compaign.MEAN(train.coupon_id)>,
 <Feature: compaign.MEAN(train.redemption_status)>,
 <Featur

In [26]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_names)


In [27]:
feature_matrix_enc.head()

,coupon_id,customer_id = 1463,customer_id = 590,customer_id = 189,customer_id = 1192,customer_id = 464,customer_id = 63,customer_id = 1378,customer_id = 1387,customer_id = 787,...,customer_demographics.MODE(customer_transaction.item_id) = 34047.0,customer_demographics.MODE(customer_transaction.item_id) = 45502.0,customer_demographics.MODE(customer_transaction.item_id) = 13174.0,customer_demographics.MODE(customer_transaction.item_id) = 30293.0,customer_demographics.MODE(customer_transaction.item_id) = 48973.0,customer_demographics.MODE(customer_transaction.item_id) = 12039.0,customer_demographics.MODE(customer_transaction.item_id) = 6958.0,customer_demographics.MODE(customer_transaction.item_id) = 18156.0,customer_demographics.MODE(customer_transaction.item_id) = 24135.0,customer_demographics.MODE(customer_transaction.item_id) is unknown
id,,,,,,,,,,,,,,,,,,,,,
1,27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,635,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,644,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,1017,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [30]:
ft.save_features(features_enc,"feature_train.json")

In [32]:
saved_features = ft.load_features('feature_train.json')
test_feat=ft.calculate_feature_matrix(saved_features, test)

KeyError: 'train'